# Deep Learning 2025/11/26：使用 LoRA 微調(大型)語言模型
- 教學目標：讓學生能使用 Hugging Face Transformers 來微調 LM
- 使用模型：google/gemma-3-270m (受限於課堂訓練時間)
- 資料集：databricks/databricks-dolly-15k

In [1]:
# 0. 設定模型與資料集名稱

MODEL_NAME = "google/gemma-3-270m"
DATASET_NAME = "databricks/databricks-dolly-15k"

In [2]:
# 導入所需套件

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from huggingface_hub import login
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training
)
import torch
from functools import partial

## 登入 HuggingFace
- 有的放在 HuggingFace 的模型在使用前需要先登入
- 我們需要先準備好 Access Token

In [ ]:
# 1. 登入 HuggingFace

login("這裡打你的登入 TOKEN")

## 資料集

In [ ]:
# 2-1. 載入 Dolly-15k dataset

dataset = load_dataset(DATASET_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 2-2. 查看資料集內容

print(dataset)

# 印出第一筆內容
for k, v in dataset["train"][0].items():
    print(f"{k}: {v}")

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})
instruction: When did Virgin Australia start operating?
context: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.
response: Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.
category: closed_qa


## 模型

In [4]:
# 3. 載入 tokenizer & 模型

# `use_fast=True` 代表使用 Rust 寫的 tokenizer，處理資料速度較快
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # load_in_8bit=True, # 可直接載入量化版本
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

In [ ]:
# 4-1. 設立指令模板 (訓練用)
# 設立大括弧，等等就能用 format 來填入

TEMPLATE_for_train = """### Instruction
{instruction}

### Context
{context}

### Answer
{response}"""

In [ ]:
# 4-2. 設立指令模板 (測試用)
# 設立大括弧，等等就能用 format 來填入

TEMPLATE_for_test = """### Instruction
{instruction}

### Answer
"""

In [ ]:
# 5. 觀察模型訓練前的行為

instruction = "What should I do on a trip to Europe?"
text = TEMPLATE_for_test.format(instruction=instruction)

device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction
What should I do on a trip to Europe?

### Answer
You probably should start with an itinerary from your hotel in the morning. It's a good idea to do this at least three times throughout your trip, but you could also do this right after the train runs out of the station. The plan can be varied depending on what type of weather you need to get out of town. Do not go anywhere too quickly. The key is to arrive at the right time and make good use of the trains. If you have your passport, take it back.


## 資料前處理

In [ ]:
# 6. 處理 prompt 格式（instruction + input + output）

def format_prompt(example, template):
    prompt = template.format(
        instruction=example["instruction"],
        context=example["context"],
        response=example["response"]
    )
    return {"text": prompt}

dataset = dataset.map(partial(format_prompt, template=TEMPLATE_for_train))

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][0]["text"])

### Instruction
When did Virgin Australia start operating?

### Context
Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.

### Answer
Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.


In [ ]:
# 7. tokenize

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

## 使用 LoRA

In [5]:
# 8-1. 查看模型中的模組名稱

model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)

In [ ]:
# 8-2. 設定 LoRA config
# "gate_proj", "up_proj", "down_proj" 屬於 GLU 的參數
# https://arxiv.org/abs/2002.05202

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

## 使用 Trainer

In [ ]:
# 9. 設定訓練參數

training_args = TrainingArguments(
    output_dir="./gemma-3-270m-dolly15k-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    max_steps=300,
    save_total_limit=1,
    report_to="none"
)

In [ ]:
# 10. Data collator (處理 padding)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# 11. 設定 Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-755182530.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
# 12. 開始訓練

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 1}.


Step,Training Loss
10,2.787400
20,2.365500
30,2.357600
40,2.362800
50,2.295000
60,2.291900
70,2.315500
80,2.263700
90,2.270300
100,2.345500


TrainOutput(global_step=300, training_loss=2.3169513893127442, metrics={'train_runtime': 254.0775, 'train_samples_per_second': 18.892, 'train_steps_per_second': 1.181, 'total_flos': 1507361764147200.0, 'train_loss': 2.3169513893127442, 'epoch': 0.31974420463629094})

In [ ]:
# 13. 觀察模型訓練後的行為

instruction = "What should I do on a trip to Europe?"
text = TEMPLATE_for_test.format(instruction=instruction)

device = "cuda"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction
What should I do on a trip to Europe?

### Answer
Europe is a huge country, but there are several things to keep in mind. First, you will want to take time to explore the different countries and to compare them. Secondly, your plans should also include a tour to each city that you visit. That is where you will want to compare things before you actually make your decision, and then pick and choose which to visit. So if you have some time on your side you can take advantage of a trip to Italy. This will allow you to get


In [ ]:
# 14. (Optional) 可以把模型上傳到 HuggingFace
# 範例 -> https://huggingface.co/yingjialin/gemma-3-270m-dolly15k-lora

# !huggingface-cli upload yingjialin/gemma-3-270m-dolly15k-lora tinyllama-dolly15k-lora